# Subway_Timetable_DB

http://subway.koreatriptips.com/subway-station.html 에서 지하철 역당 시간표를 Crawling 한다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [2]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

---

지하철 호선과 역당 시간을 저장한다. 

- function name : for_subway_timetable

In [3]:
def for_subway_timetable():
    url = "http://subway.koreatriptips.com/subway-station.html"

    # chrome webdriver 이용
    browser = webdriver.Chrome('/Users/sony/Downloads/chromedriver_win32/chromedriver')
    browser.get(url)
    time.sleep(1)

    html = browser.page_source
    dom = BeautifulSoup(html, "lxml")
    for_line = dom.select("ul.list-unstyled > li > a") # line links 받음

    conn = engine.connect()

    for i in range(12):
        for_timetable = {}  # line/station/hour/minute
        for_timetable['line'] = for_line[i].text # line 이름 저장
        href = "ul.list-unstyled > li:nth-child(" + str(i+1) + ") > a"
        browser.find_element_by_css_selector(href).click()

        html = browser.page_source
        dom = BeautifulSoup(html, "lxml")
        for_station = dom.select("ul.list-unstyled > li > a") # station links 받음

        for j in range(len(for_station)-1): # 마지막 링크는 전 페이지로 돌아감
            for_timetable['station'] = for_station[j].text # station 이름 저장
            href = "ul.list-unstyled > li:nth-child(" + str(j+1) + ") > a"
            browser.find_element_by_css_selector(href).click()

            html = browser.page_source
            dom = BeautifulSoup(html, "lxml")
            for for_direction in range(2):
                if for_direction == 0:
                    for_time = dom.select("section.content > div:nth-of-type(3) > div.box-body tbody td")
                    for_timetable['direction'] = 'U' # direction 저장
                else:
                    for_time = dom.select("section.content > div:nth-of-type(5) > div.box-body tbody td")
                    for_timetable['direction'] = 'D' # direction 저장

                for k in range(0, len(for_time), 2):
                    for_timetable['hour'] = int(for_time[k].text) # hour 저장
                    minute = for_time[k+1].text.split(',')
                    for l in range(len(minute)):
                        for_timetable['minute'] = int(minute[l]) # minute 저장
                        conn.execute(subway_timetable.insert(), for_timetable) # DB에 저장
                        print(for_timetable)
            browser.back()
        browser.back()    

subway_timetable.db 정의

In [4]:
engine = create_engine("sqlite:///subway_timetable.db", echo = True)
metadata = MetaData()

subway_timetable = Table('subway_timetable', metadata,
                        Column('id', Integer, primary_key =True),
                        Column('line', String, nullable = False),
                        Column('station', String, nullable = False),
                        Column('direction', String, nullable = False),
                        Column('hour', Integer),
                        Column('minute', Integer)
                    )

metadata.create_all(engine)

2018-08-15 23:17:02,104 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-08-15 23:17:02,114 INFO sqlalchemy.engine.base.Engine ()
2018-08-15 23:17:02,116 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-08-15 23:17:02,117 INFO sqlalchemy.engine.base.Engine ()
2018-08-15 23:17:02,119 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("subway_timetable")
2018-08-15 23:17:02,120 INFO sqlalchemy.engine.base.Engine ()
2018-08-15 23:17:02,124 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE subway_timetable (
	id INTEGER NOT NULL, 
	line VARCHAR NOT NULL, 
	station VARCHAR NOT NULL, 
	direction VARCHAR NOT NULL, 
	hour INTEGER, 
	minute INTEGER, 
	PRIMARY KEY (id)
)


2018-08-15 23:17:02,125 INFO sqlalchemy.engine.base.Engine ()
2018-08-15 23:17:02,170 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
for_subway_timetable()